In [2]:
import pandas as pd
import simpy

In [3]:
base_dados = pd.read_csv("DoaçãoRim.csv", sep=",")

display(base_dados)

,date,age_at_list_registration,age_cat,time_on_Dialysis,prior_transplant,number_prior_transplant,patient_still_on_list,date_acutal,death,Time_death,Transplant,Transplant_Y_N,X36MthsTx,Time_Tx,removed_list,razon_removed,time,event
0,29/06/2017,67,Maior.60,1.0,Não,0,Sim,43307,Não,13,Não,Não,0,1307,Não,NaN,392,0
1,03/11/2008,58,18.a.60,4.0,Não,0,Não,43307,Sim,69,Óbito Lista,Não,0,6887,Não,NaN,2066,2
2,13/07/2010,51,18.a.60,2.0,Não,0,Não,43307,Não,98,Não,Não,0,1217,Sim,Removido (suspenso > 365 dias),365,3
3,03/10/2011,52,18.a.60,17.0,Sim,1,Não,43307,Não,83,Não,Não,0,1217,Sim,Removido (suspenso > 365 dias),365,3
4,05/07/2006,67,Maior.60,68.0,Não,0,Não,43307,Sim,6,Óbito Lista,Não,0,647,Não,NaN,194,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48148,29/12/2004,62,Maior.60,14.0,Não,0,Não,43307,Sim,11,Óbito Lista,Não,0,1147,Não,NaN,344,2
48149,25/05/2004,50,18.a.60,121.0,Sim,1,Não,43307,Sim,97,Óbito Lista,Não,0,973,Não,NaN,2919,2
48150,11/02/2009,49,18.a.60,15.0,Não,0,Não,43307,Sim,41,Óbito Lista,Não,0,4123,Não,NaN,1237,2
48151,16/11/2017,83,Maior.60,26.0,Não,0,Sim,43307,Não,8,Não,Não,0,84,Não,NaN,252,0


## Mantendo apenas os dados que serão utilizados 

In [4]:
remover = base_dados.drop(columns=["age_cat","time_on_Dialysis","prior_transplant","date_acutal","Time_death","Transplant_Y_N","X36MthsTx","Time_Tx","event","razon_removed"])

remover.head()

,date,age_at_list_registration,number_prior_transplant,patient_still_on_list,death,Transplant,removed_list,time
0,29/06/2017,67,0,Sim,Não,Não,Não,392
1,03/11/2008,58,0,Não,Sim,Óbito Lista,Não,2066
2,13/07/2010,51,0,Não,Não,Não,Sim,365
3,03/10/2011,52,1,Não,Não,Não,Sim,365
4,05/07/2006,67,0,Não,Sim,Óbito Lista,Não,194


In [9]:
base_dados = pd.DataFrame(base_dados['date'])
vetor = base_dados.value_counts
print(vetor)


<bound method DataFrame.value_counts of              date
0      29/06/2017
1      03/11/2008
2      13/07/2010
3      03/10/2011
4      05/07/2006
...           ...
48148  29/12/2004
48149  25/05/2004
48150  11/02/2009
48151  16/11/2017
48152  06/05/2015

[48153 rows x 1 columns]>


method

In [51]:



def fila(env, recurso, paciente, saida, chegada, prioridade):
    yield env.timeout(chegada)
    yield env.timeout(saida)
    if saida != 0:
        print(paciente, 'saiu da fila em', saida)
    else:
        with recurso.request(priority=prioridade) as req:
            yield req
            print(paciente, 'atendida em', env.now)
            yield env.timeout(5)
            print(paciente, 'saiu do atendimento em', env.now)

env = simpy.Environment()
recurso = simpy.PriorityResource(env, capacity=1)

env.process(fila(env, recurso, 'Maria', 0, 0, 0))
env.process(fila(env, recurso, 'Paula', 2, 2, 1))
env.process(fila(env, recurso, 'Paulo', 6, 4, 1))
env.process(fila(env, recurso, 'Alana', 0, 0, 1))
env.process(fila(env, recurso, 'Sandra', 0, 0, 0))

env.run()



Maria atendida em 0
Paula saiu da fila em 2
Maria saiu do atendimento em 5
Sandra atendida em 5
Paulo saiu da fila em 6
Sandra saiu do atendimento em 10
Alana atendida em 10
Alana saiu do atendimento em 15
